## FBA simulation
Simulating the FBA part of the model with `cobrapy` to check that the FBA part is working.

In [1]:
from __future__ import print_function, division
import pandas as pd
import cobra
from sbmlutils import fbc
from pprint import pprint

In [2]:
# load model
source = './results/diauxic_fba.xml'
mfba = cobra.io.read_sbml_model(source)
cobra.io.sbml3.validate_sbml_model(source)

cobra/io/sbml3.py:249 UserWarning: loading SBML model without fbc:strict="true"


(<Model diauxic_fba at 0x7f44aa400190>,
 {'SBML errors': [], 'other': [], 'validator': [], 'warnings': []})

In [3]:
# objective function & boundaries
pprint(mfba.objective)
fbc.cobra_reaction_info(mfba)

{<Reaction v1 at 0x7f44aa3eee90>: 1.0,
 <Reaction v2 at 0x7f44aa3eeed0>: 1.0,
 <Reaction v3 at 0x7f44aa3eef10>: 1.0,
 <Reaction v4 at 0x7f44aa3eef50>: 1.0}


,lb,ub,objective
vO2,0,15,NaN
vGlcxt,0,10,NaN
vAc,-1000,1000,NaN
vX,0,1000,NaN
v1,0,1000,1
v2,0,1000,1
v3,0,1000,1
v4,0,1000,1


In [4]:
# simulate
# s = mfba.optimize()
s = mfba.optimize(objective_sense="maximize")

print(s)
print(mfba.solution.status)

<Solution 1.10 at 0x7f44aa400d90>
optimal


In [5]:
# summary of solution
mfba.summary()

IN FLUXES     OUT FLUXES    OBJECTIVES
------------  ------------  ------------
O2     15     X  1.1        v1  0.0384
Glcxt  10                   v2  1.06
Ac      1.51                v3  0
                            v4  0


In [6]:
# FVA
# running flux variability analysis with the model
fva_result = cobra.flux_analysis.flux_variability_analysis(mfba, mfba.reactions)
pd.DataFrame.from_dict(fva_result).T.round(5)

,maximum,minimum
v1,0.03836,0.03836
v2,1.05708,1.05708
v3,0.00000,0.00000
v4,0.00000,0.00000
vAc,1.51242,1.51242
vGlcxt,10.00000,10.00000
vO2,15.00000,15.00000
vX,1.09544,1.09544


In [7]:
from cobra.test import test_all